
# Projet 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd
import json

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# chargement et traitement des données
geo = pd.read_csv("/content/drive/MyDrive/correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [4]:
# Ajout colonne 'CP Ville'
geo['CP Ville'] = geo['Code Postal'] + ' ' + geo['Commune'] 
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville
Code INSEE,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114,01400 L'ABERGEMENT-CLEMENCIAT
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017,01640 L'ABERGEMENT-DE-VAREY
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926,01500 AMBERIEU-EN-BUGEY
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273,01330 AMBERIEUX-EN-DOMBES
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320,01300 AMBLEON


In [5]:
# fonction recherche de ville
def ville(lat, long):
  geo['Distance'] = np.sqrt((geo['Latitude'] - lat)**2 + (geo['Longitude'] - long)**2)
  return geo.loc[geo['Distance'].idxmin(), 'Commune']

In [6]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600
dms2dec(48, 42, 52)

48.714444444444446

In [7]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'BOSJEAN'

In [8]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'PALAISEAU'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [9]:
# Parsing de la colonne geo_shape
geo_shape = geo['geo_shape'].apply(lambda x: json.loads(x))
geo['type'] = geo_shape.apply(lambda x: x['type'])
geo['coordonees'] = geo_shape.apply(lambda x: x['coordinates'])

In [10]:
# Décompte des valeurs accédées avec la clé "type"
geo['type'].value_counts()

Polygon         36670
MultiPolygon       72
Name: type, dtype: int64

In [11]:
# Décompte des longueurs des listes accédées avec la clé "coordinates"
geo['coordonees'].str.len().value_counts()

1    36660
2       80
4        1
3        1
Name: coordonees, dtype: int64

In [12]:
# Ville la plus complexe géométriquement
# c'est la ville possédant le plus d'éléments dans 'coordinates'

geo.loc[geo['coordonees'].str.len().idxmax(), 'Commune']

'HYERES'

In [13]:
# Deuxieme ville la plus complexe géométriquement
# c'est la ville avec 3 coordonnées

geo.loc[geo['coordonees'].str.len() == 3, 'Commune']

Code INSEE
27701    VAL-DE-REUIL
Name: Commune, dtype: object

In [14]:
# Villes de type 'Polygon' dont la  longueur des listes vaut 2
geo.loc[(geo['type'] == 'Polygon') & (geo['coordonees'].str.len() == 2), 'Commune']

Code INSEE
02232               COYOLLES
05052               EYGLIERS
09042    LA BASTIDE-DE-SEROU
22209              PLOUBALAY
2A272                SARTENE
2B049              CALENZANA
48080               LANGOGNE
67486              SUNDHOUSE
68078              EGUISHEIM
71028            BEAUVERNOIS
Name: Commune, dtype: object

In [15]:
pip install shapely

In [16]:
# Pour ces villes vérifier que le premier polygone contient bien le second
from shapely.geometry import Polygon
geo_poly_2 = geo.loc[(geo['type'] == 'Polygon') & (geo['coordonees'].str.len() == 2)]
geo_poly_2['Enclave'] = geo_poly_2['coordonees'].apply(lambda x: Polygon(x[0]).contains(Polygon(x[1])))
geo_poly_2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,Distance,type,coordonees,Enclave
Code INSEE,,,,,,,,,,,,,,,,,
02232,02600,COYOLLES,AISNE,PICARDIE,Commune simple,126.0,2489.0,0.4,"49.2133089693, 3.01306173751","{""type"": ""Polygon"", ""coordinates"": [[[3.037897...",49.213309,3.013062,02600 COYOLLES,0.915153,Polygon,"[[[3.037897560658703, 49.16921270775551], [3.0...",True
05052,05600,EYGLIERS,HAUTES-ALPES,PROVENCE-ALPES-COTE D'AZUR,Commune simple,1615.0,2987.0,0.7,"44.6925244966, 6.66688741371","{""type"": ""Polygon"", ""coordinates"": [[[6.719912...",44.692524,6.666887,05600 EYGLIERS,5.976752,Polygon,"[[[6.719912606601575, 44.70392733592357], [6.7...",True
09042,09240,LA BASTIDE-DE-SEROU,ARIEGE,MIDI-PYRENEES,Chef-lieu canton,479.0,4138.0,1.0,"43.0242795037, 1.42677885373","{""type"": ""Polygon"", ""coordinates"": [[[1.423960...",43.024280,1.426779,09240 LA BASTIDE-DE-SEROU,5.748811,Polygon,"[[[1.423960819701936, 42.99550854650971], [1.4...",True
22209,22650,PLOUBALAY,COTES-D'ARMOR,BRETAGNE,Chef-lieu canton,36.0,3600.0,2.7,"48.5596688192, -2.12838362694","{""type"": ""Polygon"", ""coordinates"": [[[-2.11237...",48.559669,-2.128384,22650 PLOUBALAY,4.376954,Polygon,"[[[-2.11237709725211, 48.52896113651634], [-2....",True
2A272,20100,SARTENE,CORSE-DU-SUD,CORSE,Sous-préfecture,190.0,20182.0,3.1,"41.5718443328, 8.93538107497","{""type"": ""Polygon"", ""coordinates"": [[[8.940487...",41.571844,8.935381,20100 SARTENE,9.786051,Polygon,"[[[8.940487655060174, 41.490088793265336], [8....",True
